# 🔍 Pesquisa - Básico

Este notebook tem como objetivo apresentar **consultas básicas no Elasticsearch**.

Vamos explorar como realizar pesquisas simples utilizando a API de busca da ferramenta, com exemplos práticos e objetivos.

A ideia é mostrar como localizar documentos com base em seus campos, utilizando filtros e parâmetros comuns do dia a dia.

Para maiores, informações, acessar o [link](https://www.elastic.co/docs/solutions/search/full-text)

Para ilustrar como fazer pesquisa, importaremos os dados de algumas músicas extraidos do site [vagalume](https://www.vagalume.com.br/).

Criando o conector e o indice para salvar as músicas

In [1]:
import json
from pathlib import Path

In [2]:
#Carregando as músicas de diferentes artistas a partir de arquivos JSON
emicidade_musicas = None
emicidade_path = Path('emicida.json')
with emicidade_path.open('r', encoding='utf-8') as f:
    emicidade_musicas = json.load(f)

charlie_musicas = None
charlie_path = Path('charlie.json')
with charlie_path.open('r', encoding='utf-8') as f:
    charlie_musicas = json.load(f)

Cada objeto dentro da lista possui as seguintes chaves

letra: Letra da música

titulo: Nome da música

Autor: Autores da música

Compositor: Compositores da música

Editor: Editores da música

URL: link dentro do site vagalume

Administração: Administração da música

Ano: Ano da publicação

Extracao: Dia que os dados foram extraidos

In [3]:
emicidade_musicas[0]

{'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
 'autor': 'Emicida',
 'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angústia é eu ter calma e a vida escada  \n  Junto ler almas pra além da pressão  \n  As voz em declive na mão desse Barrabás  \n  Onde o milagre jaz  \n  Só prova a urgência de livros  \n  perante o estrago que um sábio faz  \n  O mestre em dívidas avidas  \n  Sem noção do que são dádivas  \n  No tempo onde a única que corre livre  \n  aqui sã

In [ ]:
# Criando o conector com elasticsearch
# Modificar a variável PASSWORD para a senha configurado no .env
from elasticsearch import Elasticsearch

HOST = 'http://localhost:9200'
USERNAME = 'elastic'
PASSWORD = 'test123'

client = Elasticsearch(
    HOST,
    basic_auth=(USERNAME,PASSWORD)
)


In [11]:
#Verificando a conexão com o Elasticsearch
client.info()

ObjectApiResponse({'name': '73348a783ff1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dTLY6PORQHSoQ3_mIDx4VA', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
#Criando o índice para as músicas
INDEX_MUSICAS = 'musicas_pesquisa'

if client.indices.exists(index=INDEX_MUSICAS):
    client.indices.delete(index=INDEX_MUSICAS)
client.indices.create(index=INDEX_MUSICAS)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'musicas_pesquisa'})

In [15]:
#Salvando as músicas do Emicida no Elasticsearch
for musica in emicidade_musicas:
    client.index(
        index=INDEX_MUSICAS,
        body=musica
    )

for musica in charlie_musicas:
    client.index(
        index=INDEX_MUSICAS,
        body=musica
    )

### Pesquisa básica de texto

Inicialmente, faremos algumas pesquisas básicas e entender como elastisearch retorna 
A linguagem de pesquisa chama DSL, segue o [link](https://www.elastic.co/docs/reference/query-languages/)


In [16]:
#Pesquisando pela a palavra "Emicida" no campo "autor"

response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match":{
            "autor":"Emicida"
        }
    }
)

In [18]:
response.body

{'took': 69,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 0.8713851,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 0.8713851,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angú

O Elasticsearch retorna os seguintes campos

1. 'took':  Quanto tempo para fazer a pesquisa (em milesegundos)
2. 'timed_out': Se teve timeout
3. '_shards': Se teve sucesso (1 = True) ou não
4. 'hits': Um objeto com as seguintes chaves

    4.1 total: Quantos objetos satisfazem a pesquisa (neste caso, são 25 objetos)
    
    4.2' max_score': Qual é a maior pontuação. O elastisearch informa uma pontuação para indicar a relevância do objeto perante a pesquisa. Quanto maior a pontuação, mais relevante é o documento. Para verificar o cálculo, acesse o [link](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables)

    4.3 hits: Lista dos 10 primeiros objetos e suas pontuações e id (código único) 


In [19]:
#Pesquisando pela a palavra "Manuel" no campo "autor". Não existe música com esse autor
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match":{
            "autor":"Manuel"
        }
    }
)

In [21]:
response.body

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

### Should, Must e Filter


A linguagem DSL possui 3 componentes/palavras chaves importantes: Should,Must e Filter. Cada um destes desempenha um papel fundamental para pesquisar e alterar a pontuação da pesquisa

In [ ]:
# O termo Must indica que todos os critérios devem ser atendidos
# Pesquisando pela a palavra Emicida na chave "autor"  e a palavra "Amor" na chave  "letra"
# Podemos 
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "must":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                },
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)



In [24]:
response.body

{'took': 42,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 2.7495031,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 2.7495031,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angús

In [25]:
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "must":[
                {
                    "match":{
                        "autor":"Charlie"
                    }
                },
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)

In [26]:
response.body

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10, 'relation': 'eq'},
  'max_score': 2.5110602,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'sH5WN5cBiMBQPnh-CfSf',
    '_score': 2.5110602,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Meu Novo Mundo',
     'autor': 'Charlie Brown Jr',
     'letra': 'Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto com sorriso inocen

In [27]:
# O termo Should indica que pelo menos um dos critérios deve ser atendido. Isso altera a relevância do objeto
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "should":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                }
            ]
        }
    }
)

In [28]:
response.body

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 0.8713851,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 0.8713851,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angú

Observe que retornou 25 documentos, ou seja, todas as músicas do Emicida e do Charlie, porém 
o objeto com maior pontuação é o objeto com Emicida pois satisfaz o filter 

In [29]:
#Fazendo uma combinação de termos Must e Should
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "must":[
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ],
            "should":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                }
            ]
        }
    }
)

In [31]:
response.body

{'took': 15,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 17, 'relation': 'eq'},
  'max_score': 2.7495031,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 2.7495031,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angú

Temos todas as músicas com a palavra "Amor" na letra, porém os mais relevantes são as músicas do Emicida, devido ao filter

In [32]:
# O termo Filter indica que o critério deve ser atendido, mas não influencia na relevância do objeto
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "filter":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                }
            ]
        }
    }
)

In [33]:
response.body

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 0.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 0.0,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angústia é eu te

Observe que o max_score é zero porque o filter não altera o score

In [34]:
#Fazendo uma pesquisa combianando os termos Filter e Should
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "filter":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                }
            ],
            "should":[
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)

In [35]:
response.body

{'took': 18,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 1.878118,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 1.878118,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angúst

Neste caso, somente o termo "Amor" é relevante para o calculo da relevância.

### Pesquisa por ano ou data

In [38]:
# Pesquisando por música publicado em 2015
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match":{
            "ano":2015
        }
    }
)

In [39]:
response.body

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'mH5WN5cBiMBQPnh-B_RI',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Passarinhos (Part. Vanessa da Mata)',
     'autor': 'Emicida',
     'letra': 'Despencados de voos cansativos  \n  Complicados e pensativos  \n  Machucados após tantos crivos  \n  Blindados com nossos motivos  \n  Amuados, reflexivos  \n  E dá-lhe anti-depressivos  \n  Acanhados entre discos e livros  \n  Inofensivos  \n    \n  Será que o sol sai pra um voo melhor  \n  Eu vou esperar, talvez na primavera  \n  O céu clareia e vem calor vê só  \n  O que sobrou de nós e o que já era  \n  Em colapso o planeta gira, tanta mentira  \n  Aumenta a ira de quem sofre mudo  \n  A página vira, o são delira, então a gente pira  \n  E no meio disso tudo  \n  Tamo tipo  \n  

In [42]:
#Pesquisando por música publicado em 2015 ou 2016. Como ano é um campo numérico, podemos usar o termo Range
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "range":{
            "ano":{
                "gte":2015,
                "lte":2016
            }
        }
    }
)

In [43]:
response.body

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'mH5WN5cBiMBQPnh-B_RI',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Passarinhos (Part. Vanessa da Mata)',
     'autor': 'Emicida',
     'letra': 'Despencados de voos cansativos  \n  Complicados e pensativos  \n  Machucados após tantos crivos  \n  Blindados com nossos motivos  \n  Amuados, reflexivos  \n  E dá-lhe anti-depressivos  \n  Acanhados entre discos e livros  \n  Inofensivos  \n    \n  Será que o sol sai pra um voo melhor  \n  Eu vou esperar, talvez na primavera  \n  O céu clareia e vem calor vê só  \n  O que sobrou de nós e o que já era  \n  Em colapso o planeta gira, tanta mentira  \n  Aumenta a ira de quem sofre mudo  \n  A página vira, o são delira, então a gente pira  \n  E no meio disso tudo  \n  Tamo tipo  \n   

In [40]:
#Pesquisando por música publicado em 2015 ou 2016 e com a palavra "Amor" na letra
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "filter":[
                {
                    "range":{
                        "ano":{
                            "gte":2015,
                            "lte":2016
                        }
                    }
                }
            ],
            "should":[
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)

In [41]:
response.body

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8, 'relation': 'eq'},
  'max_score': 1.4900708,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'pH5WN5cBiMBQPnh-CPRz',
    '_score': 1.4900708,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Casa',
     'autor': 'Emicida',
     'letra': 'Lá fora é selva, a sós entre luz e trevas  \n  Noiz presos nessas fases de guerra, medo e monstros  \n  Tipo "Jogos Vorazes"  \n    \n  É pau, é pedra, é míssil  \n  E crer é cada vez mais difícil  \n  Entende um negócio, nunca foi fácil  \n  Solo não dócil, esperança fóssil  \n    \n  O samba deu conselhos, ouça  \n  Jacaré que dorme vira bolsa  \n  Amor, eu disse no começo  \n  É quem tem valor versus quem tem preço  \n    \n  Segue teu instinto  \n  Que ainda é Deus e o Diabo na terra do Sol  \n  Onde a felicidade se pisca, é isca  \n  E a realidade trisca, anzol  \n  Corre!  \n    \n  O céu é meu p

### Ajustando paramêtros de pesquisa

A busca possui outros parâmetros além do campo query:

O parâmetro size define a quantidade de objetos a serem retornados em hits.

O parâmetro from determina a partir de qual objeto os resultados devem começar a ser exibidos (utilizado para paginação).

O parâmetro _source especifica quais chaves dos objetos devem ser incluídas na resposta.

O parâmetro sort define a ordem de exibição dos resultados, desconsiderando o cálculo de relevância.

In [49]:
# Retornando os primeiros 15 objetos

response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=15
)

In [50]:
response.body

{'took': 22,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angústia é eu te

In [51]:
# Retornando os proximos 15 objetos, ou seja, os objetos 16 a 30
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=15,
    from_=15
)

In [52]:
response.body

{'took': 25,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'pn5WN5cBiMBQPnh-CPSm',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Hoje Cedo (Part. Pitty)',
     'autor': 'Emicida',
     'letra': 'Hoje cedo  \n  Quando eu acordei e não te vi  \n  Eu pensei em tanta coisa  \n  Tive medo  \n  Ah, como eu chorei  \n  Eu sofri  \n  Em segredo  \n  Tudo isso  \n  Hoje cedo  \n    \n  Holofotes fortes, purpurina  \n  E os sorrisos dessas "mina" só me lembram cocaína  \n  Em cinco abrem-se cortinas  \n  Estáticas retinas brilham, garoa fina.  \n    \n  Que fita  \n  Meus poema me trouxe  \n  Onde eles não habita.  \n    \n  A fama irrita, grana dita, cê desacredita  \n  Fantoches, pique Celso Pitta mentem  \n  Mortos tipo meu pai, nem eu me sinto presente.  \n    \n  Aí, é rima que "cês qué"? 

In [ ]:
#Retornado somente o campo "autor". Reduzindo o tamanho de dados carregados 
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=3,
    _source=["autor"]
)

In [48]:
response.body

{'took': 118,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'autor': 'Emicida'}},
   {'_index': 'musicas_pesquisa',
    '_id': 'mH5WN5cBiMBQPnh-B_RI',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'autor': 'Emicida'}},
   {'_index': 'musicas_pesquisa',
    '_id': 'mX5WN5cBiMBQPnh-B_R4',
    '_score': 1.0,
    '_ignored': ['letra.keyword'],
    '_source': {'autor': 'Emicida'}}]}}

In [53]:
# Ordernando pelo o ano
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=5,
    sort=[
        {"ano": {"order": "asc"}}
    ]
)

In [54]:
response.body

{'took': 307,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'v35WN5cBiMBQPnh-CvS0',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Aquela Paz',
     'autor': 'Charlie Brown Jr',
     'letra': 'A vida é feita de atitudes nem sempre decentes  \n  Não lhe julgam pela razão, mas pelos seus antecedentes  \n  É quando eu volto a me lembrar do que eu pensava nem ter feito  \n  Vem, me traz aquela paz  \n  Você procura a perfeição, eu tenho andado sobre efeito  \n  Mas posso te dizer que já não aguento mais  \n  Desencana, não vou mudar por sua causa, não tem jeito  \n  Quem é que decide o que é melhor pra minha vida agora?  \n    \n  Ouvi dizer que só era triste quem queria  \n  Ouvi dizer que só era triste quem queria  \n  Ouvi dizer que só era triste quem queria  \n  Ouvi dizer que só era

In [61]:
#Retornando todas as musicas através da páginando usando from
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=5,
    from_=0,
    _source=["autor", "ano","titulo"]
)

total = response['hits']['total']['value']
musicas = [i['_source'] for i in response['hits']['hits']]

while len(musicas) < total:
    response = client.search(
        index=INDEX_MUSICAS,
        query={
            "match_all":{}
        },
        size=5,
        from_=len(musicas),
        _source=["autor", "ano","titulo"]
    )
    
    musicas.extend([i['_source'] for i in response['hits']['hits']])

In [62]:
musicas

[{'ano': 2019,
  'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
  'autor': 'Emicida'},
 {'ano': 2015,
  'titulo': 'Passarinhos (Part. Vanessa da Mata)',
  'autor': 'Emicida'},
 {'ano': 2015,
  'titulo': 'Levanta e Anda (Part. Rael da Rima)',
  'autor': 'Emicida'},
 {'ano': 2019,
  'titulo': 'Pequenas Alegrias da Vida Adulta',
  'autor': 'Emicida'},
 {'ano': 2019,
  'titulo': 'Quem Tem Um Amigo (Tem Tudo) (Part. Tokyo Ska Paradise Orchestra e Os Prettos & Zeca Pagodinho)',
  'autor': 'Emicida'},
 {'ano': 2019,
  'titulo': 'Ismália (Part. Larissa Luz & Fernanda Montenegro)',
  'autor': 'Emicida'},
 {'ano': 2015, 'titulo': 'Triunfo', 'autor': 'Emicida'},
 {'ano': 2018,
  'titulo': 'Mãe (Part. Dona Jacira e Anna Tréa)',
  'autor': 'Emicida'},
 {'ano': 2011, 'titulo': 'Velhos Amigos', 'autor': 'Emicida'},
 {'ano': 2019,
  'titulo': 'A Ordem Natural das Coisas (Part. MC Tha)',
  'autor': 'Emicida'},
 {'ano': 2018, 'titulo': 'Oásis (feat. Miguel)', 

In [63]:
len(musicas)

50

In [64]:
len(musicas) == total

True

### Paginação

O elastisearch possui a funcionalidade de scroll infinito

In [73]:
#Fazendo scroll infinito
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match_all":{}
    },
    size=3,
    scroll='2h', # Tempo de vida do scroll. Neste caso fica por 2 horas. Acesse o link https://www.elastic.co/guide/en/elasticsearch/reference/8.18/api-conventions.html#time-units
    _source=["autor", "ano","titulo"],
    sort=[
        {"ano": {"order": "asc"}}
    ]
)

In [74]:
response.body

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFmFjZ1RPWG9HU2dteVpWbVRtZ1g3VXcAAAAAAAAAPxZNRHZVQzdhdFNscVlLWVg1cEMtNmxR',
 'took': 18,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'v35WN5cBiMBQPnh-CvS0',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 1997,
     'titulo': 'Aquela Paz',
     'autor': 'Charlie Brown Jr'},
    'sort': [1997]},
   {'_index': 'musicas_pesquisa',
    '_id': 'uH5WN5cBiMBQPnh-CvQ0',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 2004,
     'titulo': 'Vícios e Virtudes',
     'autor': 'Charlie Brown Jr'},
    'sort': [2004]},
   {'_index': 'musicas_pesquisa',
    '_id': 'w35WN5cBiMBQPnh-C_QF',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 2004,
     'titulo': 'Só Por Uma Noite',
     

A resposta possui um campo novo

1._scroll_id : Id do scroll. Para obtér a proxima página, é necessário o id do scroll

In [75]:
#Pegando o proximo scroll
scroll_id = response['_scroll_id']  
response = client.scroll(
    scroll_id=scroll_id,
    scroll='2h' # Tempo de vida do scroll. Neste caso fica por 2 horas. Acesse o link https://www.elastic.co/guide/en/elasticsearch/reference/8.18/api-conventions.html#time-units
)

In [76]:
response.body

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFmFjZ1RPWG9HU2dteVpWbVRtZ1g3VXcAAAAAAAAAPxZNRHZVQzdhdFNscVlLWVg1cEMtNmxR',
 'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'vn5WN5cBiMBQPnh-CvSi',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 2005,
     'titulo': 'I Feel So Good Today',
     'autor': 'Charlie Brown Jr'},
    'sort': [2005]},
   {'_index': 'musicas_pesquisa',
    '_id': 'wH5WN5cBiMBQPnh-CvTH',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 2005,
     'titulo': 'Tamo Aí Na Atividade',
     'autor': 'Charlie Brown Jr'},
    'sort': [2005]},
   {'_index': 'musicas_pesquisa',
    '_id': 'n35WN5cBiMBQPnh-CPQH',
    '_score': None,
    '_ignored': ['letra.keyword'],
    '_source': {'ano': 2011, 'titulo': 'Velhos Amigos', 